# Machine Learning

In this file, instructions how to approach the challenge can be found.

We are going to work on different types of Machine Learning problems:

- **Regression Problem**: The goal is to predict delay of flights.
- **(Stretch) Multiclass Classification**: If the plane was delayed, we will predict what type of delay it is (will be).
- **(Stretch) Binary Classification**: The goal is to predict if the flight will be cancelled.

## Main Task: Regression Problem

The target variable is **ARR_DELAY**. We need to be careful which columns to use and which don't. For example, DEP_DELAY is going to be the perfect predictor, but we can't use it because in real-life scenario, we want to predict the delay before the flight takes of --> We can use average delay from earlier days but not the one from the actual flight we predict.  

For example, variables **CARRIER_DELAY, WEATHER_DELAY, NAS_DELAY, SECURITY_DELAY, LATE_AIRCRAFT_DELAY** shouldn't be used directly as predictors as well. However, we can create various transformations from earlier values.

We will be evaluating your models by predicting the ARR_DELAY for all flights **1 week in advance**.

### Feature Engineering

Feature engineering will play a crucial role in this problems. We have only very little attributes so we need to create some features that will have some predictive power.

- weather: we can use some weather API to look for the weather in time of the scheduled departure and scheduled arrival.
- statistics (avg, mean, median, std, min, max...): we can take a look at previous delays and compute descriptive statistics
- airports encoding: we need to think about what to do with the airports and other categorical variables
- time of the day: the delay probably depends on the airport traffic which varies during the day.
- airport traffic
- unsupervised learning as feature engineering?
- **what are the additional options?**: Think about what we could do more to improve the model.

In [52]:
import pandas as pd
import numpy as np

In [53]:
flights = pd.read_csv('flights_rand.csv')

In [54]:
flights1 = flights.drop(['dep_time', 'actual_elapsed_time', 'air_time','dep_delay', 'carrier_delay', 'weather_delay', 'nas_delay', 'security_delay', 'late_aircraft_delay', 'taxi_out', 'wheels_off', 'wheels_on', 'taxi_in', 'arr_time', 'origin', 'branded_code_share', 'carrier_delay','weather_delay','nas_delay','security_delay', 'late_aircraft_delay', 'first_dep_time', 'total_add_gtime','longest_add_gtime','cancelled', 'cancellation_code', 'diverted', 'dup', 'flights', 'no_name'], axis = 1)

In [55]:
tail_number_mode = flights1['tail_num'].mode()

In [56]:
flights1['tail_num'] = flights1['tail_num'].fillna(str(tail_number_mode[0]))

In [57]:
flights1['arr_delay'] = flights1['arr_delay'].fillna(flights1['arr_delay'].mean())

* make the types categories

In [58]:
flights1["mkt_carrier"] = flights1["mkt_carrier"].astype("category")
flights1["op_unique_carrier"] = flights1["op_unique_carrier"].astype("category")
flights1["tail_num"] = flights1["tail_num"].astype("category")
flights1["op_carrier_fl_num"] = flights1["op_carrier_fl_num"].astype("category")
flights1["origin_airport_id"] = flights1["origin_airport_id"].astype("category")
flights1["dest_airport_id"] = flights1["dest_airport_id"].astype("category")
flights1["mkt_carrier_fl_num"] = flights1["mkt_carrier_fl_num"].astype("category")

* make arr_delay categorial

In [59]:
flights1['arr_delay_cat'] = flights1['arr_delay'].apply(lambda x: 1 if x > 0 else 0)

In [60]:
# ENCODE AIRPORTS AND TAILNUM
from sklearn.preprocessing import OrdinalEncoder
encoder = OrdinalEncoder()
flights1['mkt_carrier'] = encoder.fit_transform(flights1[['mkt_carrier']])
flights1['mkt_unique_carrier'] = encoder.fit_transform(flights1[['mkt_unique_carrier']])
flights1['mkt_carrier_fl_num'] = encoder.fit_transform(flights1[['mkt_carrier_fl_num']])
flights1['op_unique_carrier'] = encoder.fit_transform(flights1[['op_unique_carrier']])
flights1['tail_num'] = encoder.fit_transform(flights1[['tail_num']])
flights1['op_carrier_fl_num'] = encoder.fit_transform(flights1[['op_carrier_fl_num']])
flights1['origin_airport_id'] = encoder.fit_transform(flights1[['origin_airport_id']])
flights1['dest_airport_id'] = encoder.fit_transform(flights1[['dest_airport_id']])

* adding month, day of the week, day of the month

In [61]:
flights1['fl_date'] = pd.to_datetime(flights1['fl_date'], errors='coerce')
flights1['month'] = flights1['fl_date'].dt.month
flights1['day_of_week'] = flights1['fl_date'].dt.dayofweek
flights1['day_of_month'] = flights1['fl_date'].dt.day
flights1['year'] = flights1['fl_date'].dt.year

In [62]:
flights1 = flights1.drop('fl_date', axis = 1)

In [65]:
flights1 = flights1.drop(['origin_city_name', 'dest', 'dest_city_name'], axis = 1)

### Feature Selection / Dimensionality Reduction

We need to apply different selection techniques to find out which one will be the best for our problems.

- Original Features vs. PCA conponents?

* PCA testing

In [458]:
'''from sklearn.decomposition import PCA
import matplotlib.pyplot as plt'''

'from sklearn.decomposition import PCA\nimport matplotlib.pyplot as plt'

### Modeling

Use different ML techniques to predict each problem.

- linear
- Naive Bayes
- Random Forest Regressor
- SVM classification
- XGBoost regresspr
- The ensemble of your own choice

#### pickle module to save model

In [66]:
import pickle

* target and features

In [67]:
y = np.array(flights1.arr_delay)

In [68]:
y_cat = np.array(flights1.arr_delay_cat)

In [69]:
X_df = flights1.drop(['arr_delay', 'arr_delay_cat'], axis = 1)

In [70]:
X = np.array(X_df)

* train test split and making samples

In [71]:
import sklearn.model_selection as model_selection
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, train_size=0.70,test_size=0.30, random_state=101, shuffle = True)

In [72]:
#creating y_train_cat and y_test_cat
X_train, X_test, y_train_cat, y_test_cat = model_selection.train_test_split(X, y_cat, train_size=0.70,test_size=0.30, random_state=101, shuffle = True)

In [73]:
X_train_sample = pd.DataFrame(X_train).sample(frac = 0.1).values

In [74]:
y_train_sample = pd.DataFrame(y_train).sample(frac = 0.1).values

In [75]:
y_trainCat_sample = pd.DataFrame(y_train_cat).sample(frac = 0.1).values

In [76]:
X_test_sample = pd.DataFrame(X_test).sample(frac = 0.1).values

* Scaling

In [77]:
'''scaler = StandardScaler()
X = scaler.fit_transform(X)'''

'scaler = StandardScaler()\nX = scaler.fit_transform(X)'

In [78]:
'''y = (y - y.mean()) / y.std()'''

'y = (y - y.mean()) / y.std()'

In [79]:
from sklearn.metrics import r2_score

* Scaling PIPE

In [80]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

In [81]:
pipe = Pipeline([('scaler', StandardScaler())])
pipe.fit(X_train, y_train)

Pipeline(steps=[('scaler', StandardScaler())])

#### Linear

In [82]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV

In [83]:
#logistic regression
linear_model = LinearRegression()

In [84]:
linear_pipe = Pipeline(steps=[('scaler', StandardScaler()), ('linear_model', LinearRegression())])

In [85]:
linear_pipe.fit(X_train, y_train)

Pipeline(steps=[('scaler', StandardScaler()),
                ('linear_model', LinearRegression())])

In [86]:
y_pred_linear = linear_pipe.predict(X_test)

In [87]:
r2_score = linear_pipe.score(X_test, y_test)

In [88]:
r2_score

0.0070668570929973695

* pickling

In [90]:
with open('model_linear_pickle', 'wb') as linear_file:
    pickle.dump(linear_pipe, linear_file)

In [91]:
with open('model_linear_pickle', 'rb') as linear_file:
    model_linear = pickle.load(linear_file)

#### Naives Bayes, GaussianNB Naive Bayes

In [92]:
from sklearn.naive_bayes import GaussianNB

In [93]:
NB_Gauss_model = GaussianNB()

In [94]:
NB_Gauss_pipe = Pipeline(steps=[('scaler', StandardScaler()), ('NB_Gauss_model', GaussianNB())])

In [95]:
NB_Gauss_pipe.fit(X_train, y_train_cat)

Pipeline(steps=[('scaler', StandardScaler()), ('NB_Gauss_model', GaussianNB())])

In [96]:
y_pred_NBGauss = NB_Gauss_pipe.predict(X_test)

In [97]:
r2_score = NB_Gauss_pipe.score(X_test, y_test_cat)
r2_score

0.6166666666666667

* pickling

In [440]:
with open('model_Gaussian_pickle', 'wb') as Gaussian_file:
    pickle.dump(NB_Gauss_pipe, Gaussian_file)

In [441]:
with open('model_Gaussian_pickle', 'rb') as Gaussian_file:
    model_Gaussian = pickle.load(Gaussian_file)

#### Forrest Classifier

In [322]:
from sklearn.ensemble import RandomForestClassifier

In [361]:
random_forest_class = RandomForestClassifier()

In [362]:
hyperparameters ={
    'rand_forrestClass__n_estimators': [8000, 10000],
    'rand_forrestClass__max_depth': [10],
    'rand_forrestClass__bootstrap': [False]
}

In [363]:
Forest_class_pipe = Pipeline(steps=[('scaler', StandardScaler()), ('rand_forrestClass', RandomForestClassifier())])
forest_class_grid = GridSearchCV(estimator=Forest_class_pipe, param_grid=hyperparameters, scoring = 'r2', verbose=0, cv= 5)

* test with sample to find best hyperparameters

In [154]:
rand_forrestClass_grid = GridSearchCV(estimator=random_forest_class, param_grid=hyperparameters, scoring = 'r2', verbose=0, cv= 5)

In [ ]:
forest_class_grid.fit(X_train_sample, y_trainCat_sample.ravel())
y_pred_randFor_class = forest_class_grid.predict(X_test_sample)

In [ ]:
forest_class_grid.best_estimator_

* adjusted to best hyperparameters

In [ ]:
forest_class_grid.fit(X_train, y_train_cat.ravel())

In [368]:
y_pred_randFor_class = forest_class_grid.predict(X_test)

In [ ]:
forest_class_grid.best_estimator_

In [375]:
r2_score = forest_class_grid.score(X_test, y_test_cat)
r2_score

-0.5616250137111254

* pickling

In [380]:
with open('model_Forrest_class_pickle', 'wb') as forest_class_file:
    pickle.dump(forest_class_grid, forest_class_file)

In [442]:
with open('model_Forrest_class_pickle', 'rb') as forest_class_file:
    model_forrestClass = pickle.load(forest_class_file)

#### Random Forest Regressor

In [390]:
from sklearn.ensemble import RandomForestRegressor

In [391]:
random_forest_regression_model = RandomForestRegressor()

In [392]:
hyperparameters = {
    'rand_forrestReg__n_estimators': [1000],
    'rand_forrestReg__max_depth': [2],
    'rand_forrestReg__min_samples_split':[6], 
    'rand_forrestReg__bootstrap':[True],
    'rand_forrestReg__criterion' :['mse']
}

In [393]:
Forest_reg_pipe = Pipeline(steps=[('scaler', StandardScaler()), ('rand_forrestReg', RandomForestRegressor())])
forest_reg_grid = GridSearchCV(estimator=Forest_reg_pipe, param_grid=hyperparameters, scoring = 'r2', verbose=0, cv= 5)

* test with sample to find best hyperparameters

In [386]:
'''forest_reg_grid.fit(X_train_sample, y_train_sample.ravel())
y_pred_randFor_class = forest_reg_grid.predict(X_test_sample)'''

In [387]:
forest_reg_grid.best_estimator_

Pipeline(steps=[('scaler', StandardScaler()),
                ('rand_forrestReg',
                 RandomForestRegressor(max_depth=2, min_samples_split=6,
                                       n_estimators=1000))])

* adjusted to best hyperparameters

In [394]:
forest_reg_grid.fit(X_train, y_train)
y_pred_randFo = forest_reg_grid.predict(X_test)

In [395]:
forest_reg_grid.best_estimator_

Pipeline(steps=[('scaler', StandardScaler()),
                ('rand_forrestReg',
                 RandomForestRegressor(max_depth=2, min_samples_split=6,
                                       n_estimators=1000))])

In [398]:
r2_score = forest_reg_grid.score(X_test, y_test)

In [399]:
r2_score

-0.004472808329518907

* pickling

In [400]:
with open('model_Forrest_reg_pickle', 'wb') as forest_reg_file:
    pickle.dump(forest_reg_grid, forest_reg_file)

In [401]:
with open('model_Forrest_reg_pickle', 'rb') as forest_reg_file:
    model_Forrest_reg_pickle = pickle.load(forest_reg_file)

#### SVM classification

In [402]:
from sklearn.svm import SVC, SVR, LinearSVC

In [403]:
model_svm_class = SVC()

In [408]:
hyperparameters = {'model_svm_class__kernel': ['linear'],
                   'model_svm_class__C':[1],
                   'model_svm_class__degree':[1]
}
SVC_grid_class = GridSearchCV(estimator=model_svm_class, param_grid=hyperparameters, scoring = 'r2', verbose=0, cv= 5)

In [409]:
svm_class_pipe = Pipeline(steps=[('scaler', StandardScaler()), ('model_svm_class', SVC())])
smv_class_grid = GridSearchCV(estimator=svm_class_pipe, param_grid=hyperparameters, scoring = 'r2', verbose=0, cv= 5)

* test with sample to find best hyperparameters

In [406]:
smv_class_grid.fit(X_train_sample, y_trainCat_sample.ravel())
y_pred_svm = smv_class_grid.predict(X_test_sample)

In [407]:
smv_class_grid.best_estimator_

Pipeline(steps=[('scaler', StandardScaler()),
                ('model_svm_class', SVC(C=1, degree=1, kernel='linear'))])

* adjusted to best hyperparameters

In [410]:
smv_class_grid.fit(X_train,y_train_cat)
y_pred_svc_cat = smv_class_grid.predict(X_test)

In [412]:
smv_class_grid.best_estimator_

Pipeline(steps=[('scaler', StandardScaler()),
                ('model_svm_class', SVC(C=1, degree=1, kernel='linear'))])

In [414]:
r2_score = smv_class_grid.score(X_test, y_test_cat)

In [415]:
r2_score

-0.5673981191222572

* pickling

In [416]:
with open('model_smv_class_pickle', 'wb') as smv_class_file:
    pickle.dump(smv_class_grid, smv_class_file)

In [439]:
with open('model_smv_class_pickle', 'rb') as smv_class_file:
    model_smv_class_pickle = pickle.load(smv_class_file)

#### XGBoost regressor

In [423]:
import xgboost as xgb

In [419]:
xg_reg = xgb.XGBRegressor()

In [431]:
hyperparameters = {
    'xg_reg__objective' : ['reg:squarederror'],
    'xg_reg__colsample_bytree':[0.1],
    'xg_reg__n_estimators': [3000],
    'xg_reg__max_depth': [4],
    'xg_reg__learning_rate': [0.0001],
    'xg_reg__alpha': [5]
}

In [425]:
svm_class_pipe = Pipeline(steps=[('scaler', StandardScaler()), ('xg_reg', xgb.XGBRegressor())])
xgb_grid = GridSearchCV(estimator=svm_class_pipe, param_grid=hyperparameters, scoring = 'r2', verbose=0, cv= 5)

* test with sample to find best hyperparameters

In [429]:
xgb_grid.fit(X_train_sample, y_train_sample.ravel())
y_pred_xbg = xgb_grid.predict(X_test_sample)

In [430]:
xgb_grid.best_estimator_

Pipeline(steps=[('scaler', StandardScaler()),
                ('xg_reg',
                 XGBRegressor(alpha=5, base_score=0.5, booster='gbtree',
                              colsample_bylevel=1, colsample_bynode=1,
                              colsample_bytree=0.1, gamma=0, gpu_id=-1,
                              importance_type='gain',
                              interaction_constraints='', learning_rate=0.0001,
                              max_delta_step=0, max_depth=4, min_child_weight=1,
                              missing=nan, monotone_constraints='()',
                              n_estimators=3000, n_jobs=8, num_parallel_tree=1,
                              random_state=0, reg_alpha=5, reg_lambda=1,
                              scale_pos_weight=1, subsample=1,
                              tree_method='exact', validate_parameters=1,
                              verbosity=None))])

* adjusted to best hyperparameters

In [432]:
xgb_grid.fit(X_train,y_train)
y_pred_xbg = xgb_grid.predict(X_test)

In [434]:
xgb_grid.best_estimator_

Pipeline(steps=[('scaler', StandardScaler()),
                ('xg_reg',
                 XGBRegressor(alpha=30, base_score=0.5, booster='gbtree',
                              colsample_bylevel=1, colsample_bynode=1,
                              colsample_bytree=0.1, gamma=0, gpu_id=-1,
                              importance_type='gain',
                              interaction_constraints='', learning_rate=0.005,
                              max_delta_step=0, max_depth=2, min_child_weight=1,
                              missing=nan, monotone_constraints='()',
                              n_estimators=500, n_jobs=8, num_parallel_tree=1,
                              random_state=0, reg_alpha=30, reg_lambda=1,
                              scale_pos_weight=1, subsample=1,
                              tree_method='exact', validate_parameters=1,
                              verbosity=None))])

In [436]:
r2_score = xgb_grid.score(X_test, y_test)
r2_score

0.0011911026682224213

* pickling

In [437]:
with open('model_xgb_grid_pickle', 'wb') as xgb_reg_file:
    pickle.dump(xgb_grid, xgb_reg_file)

In [438]:
with open('model_xgb_grid_pickle', 'rb') as xgb_reg_file:
    model_xgb_grid_pickle = pickle.load(xgb_reg_file)

### Evaluation

You have data from 2018 and 2019 to develop models. Use different evaluation metrics for each problem and compare the performance of different models.

You are required to predict delays on **out of sample** data from **first 7 days (1st-7th) of January 2020** and to share the file with LighthouseLabs. Sample submission can be found in the file **_sample_submission.csv_**

======================================================================
## Stretch Tasks

### Multiclass Classification

The target variables are **CARRIER_DELAY, WEATHER_DELAY, NAS_DELAY, SECURITY_DELAY, LATE_AIRCRAFT_DELAY**. We need to do additional transformations because these variables are not binary but continuos. For each flight that was delayed, we need to have one of these variables as 1 and others 0.

It can happen that we have two types of delays with more than 0 minutes. In this case, take the bigger one as 1 and others as 0.

### Binary Classification

The target variable is **CANCELLED**. The main problem here is going to be huge class imbalance. We have only very little cancelled flights with comparison to all flights. It is important to do the right sampling before training and to choose correct evaluation metrics.